# xử lý dữ liệu

## Làm sạch dữ liệu

In [14]:
import re, unicodedata
import pandas as pd

INPUT_TWO_COLS = r"/Users/huy/Documents/Hutech/HK1A 2025-2026/social-networking/project_1/phim_dienvien.csv"
MOVIE_COL = "Tên phim"
CAST_COL  = "Diễn viên"

def norm_ws(s: str) -> str:
    return re.sub(r"\s+", " ", str(s) if s is not None else "").strip()

def strip_accents(text: str) -> str:
    if not isinstance(text, str):
        text = str(text) if pd.notna(text) else ""
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
    return unicodedata.normalize("NFKC", text)

def normalize_key(name: str) -> str:
    """Tạo khóa gộp trùng: bỏ dấu + hạ chuẩn chữ, bỏ dấu chấm lửng, chuẩn khoảng trắng."""
    s = strip_accents(str(name)).casefold().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[.…]+$", "", s).strip(" ,;")
    return s

def split_cast_better(text: str):
    """
    Quy tắc:
      - 'FAPtv (Huỳnh Phương, Thái Vũ, Vinh Râu, ...)' -> ['Huỳnh Phương','Thái Vũ','Vinh Râu']
      - 'Lương Anh Vũ (Alvin Lu)' -> ['Lương Anh Vũ']  (bỏ alias)
      - Tách theo ',' hoặc ';' bình thường.
      - Bỏ '…'/'...' và khoảng trắng/dấu thừa.
    """
    if text is None or not str(text).strip():
        return []
    s = norm_ws(text)
    s = re.sub(r"[.…]+$", "", s).strip()

    raw_tokens = [t.strip(" ,;") for t in re.split(r"[;,]", s) if t.strip(" ,;")]
    out = []
    for tok in raw_tokens:
        m = re.match(r"^(.*?)(\(([^()]*)\))$", tok)  # phần_trước (bên_trong)
        if m:
            before = norm_ws(m.group(1))
            inside = norm_ws(m.group(3))
            if inside.count(",") >= 1:
                # group-list -> lấy các thành viên bên trong
                members = [norm_ws(x) for x in inside.split(",") if norm_ws(x)]
                members = [re.sub(r"[.…]+$", "", x).strip(" ,;") for x in members]
                out.extend(members)
            else:
                # alias -> giữ phần trước ngoặc
                if before:
                    out.append(before)
                elif inside:
                    out.append(inside)
        else:
            out.append(tok)

    out = [norm_ws(re.sub(r"[.…]+$", "", x)).strip(" ,;") for x in out]
    out = [x for x in out if x]

    # unique theo thứ tự xuất hiện trong CÙNG 1 phim
    seen = set(); dedup = []
    for x in out:
        if x not in seen:
            seen.add(x); dedup.append(x)
    return dedup

# ---- đọc file & chuẩn tên cột ----
df = pd.read_csv(INPUT_TWO_COLS, sep=None, engine="python", encoding="utf-8-sig")
df.columns = [norm_ws(c.replace("\ufeff","")) for c in df.columns]
assert MOVIE_COL in df.columns and CAST_COL in df.columns, "Thiếu cột bắt buộc."

# ---- tạo bảng dài phim–diễn viên ----
rows = []
for _, row in df[[MOVIE_COL, CAST_COL]].iterrows():
    movie = norm_ws(row[MOVIE_COL])
    for actor in split_cast_better(row[CAST_COL]):
        rows.append({"movie_name": movie, "actor_name": norm_ws(actor)})

long_df = pd.DataFrame(rows)
# khóa chuẩn hóa để gộp trùng
long_df["movie_key"] = long_df["movie_name"].map(normalize_key)
long_df["actor_key"] = long_df["actor_name"].map(normalize_key)

# loại trùng trong cùng 1 phim (sau chuẩn hóa)
long_df = long_df.drop_duplicates(subset=["movie_key","actor_key"]).reset_index(drop=True)

# ---- sinh ID ổn định cho phim & diễn viên ----
movies_df = (long_df[["movie_key","movie_name"]]
             .drop_duplicates("movie_key")
             .sort_values("movie_name")
             .reset_index(drop=True))
movies_df["movie_id"] = [f"M{idx+1:06d}" for idx in range(len(movies_df))]
movies_df = movies_df[["movie_id","movie_name","movie_key"]]

actors_df = (long_df[["actor_key","actor_name"]]
             .drop_duplicates("actor_key")
             .sort_values("actor_name")
             .reset_index(drop=True))
actors_df["actor_id"] = [f"A{idx+1:06d}" for idx in range(len(actors_df))]
actors_df = actors_df[["actor_id","actor_name","actor_key"]]

movies_actors_df = (long_df.merge(movies_df[["movie_id","movie_key"]], on="movie_key", how="left")
                           .merge(actors_df[["actor_id","actor_key"]], on="actor_key", how="left")
                           [["movie_id","actor_id"]]
                           .dropna()
                           .drop_duplicates()
                           .reset_index(drop=True))

print("== TÓM TẮT BƯỚC 2 ==")
print(f"Số phim: {len(movies_df)} | Số diễn viên: {len(actors_df)} | Số dòng phim–diễn viên: {len(movies_actors_df)}")
display(df.head(3))
display(long_df.head(5))
display(movies_df.head(5))
display(actors_df.head(5))
display(movies_actors_df.head(5))

== TÓM TẮT BƯỚC 2 ==
Số phim: 176 | Số diễn viên: 569 | Số dòng phim–diễn viên: 1041


,Tên phim,Diễn viên
0,Đôi mắt âm dương,"Thu Trang, Quốc Trường, Bảo Thanh, NSND Ngọc G..."
1,30 chưa phải tết,"NSND Việt Anh, NSND Hồng Vân, Trường Giang, Mạ..."
2,Gái già lắm chiêu 3,"Ninh Dương Lan Ngọc, NSND Hồng Vân, NSND Lê Kh..."


,movie_name,actor_name,movie_key,actor_key
0,Đôi mắt âm dương,Thu Trang,đoi mat am duong,thu trang
1,Đôi mắt âm dương,Quốc Trường,đoi mat am duong,quoc truong
2,Đôi mắt âm dương,Bảo Thanh,đoi mat am duong,bao thanh
3,Đôi mắt âm dương,NSND Ngọc Giàu,đoi mat am duong,nsnd ngoc giau
4,Đôi mắt âm dương,Trung Dân,đoi mat am duong,trung dan


,movie_id,movie_name,movie_key
0,M000001,1990,1990
1,M000002,30 chưa phải tết,30 chua phai tet
2,M000003,578: Phát đạn của kẻ điên,578: phat đan cua ke đien
3,M000004,Ai thương ai mến,ai thuong ai men
4,M000005,"B4S – Trước giờ ""Yêu""","b4s – truoc gio ""yeu"""


,actor_id,actor_name,actor_key
0,A000001,A Tới,a toi
1,A000002,Amee,amee
2,A000003,Anh Dũng,anh dung
3,A000004,Anh Phạm,anh pham
4,A000005,Anh Thư,anh thu


,movie_id,actor_id
0,M000169,A000425
1,M000169,A000385
2,M000169,A000035
3,M000169,A000259
4,M000169,A000449


In [15]:

# Đếm số dòng NA và trùng
na_rows = movies_actors_df.isna().any(axis=1).sum()
dup_rows = len(movies_actors_df) - len(movies_actors_df.drop_duplicates(subset=['movie_id','actor_id']))

print(f"- Dòng có NA: {na_rows}")
print(f"- Dòng trùng: {dup_rows}")
print(f"- Tổng số quan hệ phim–diễn viên: {len(movies_actors_df)}")

# Tạo bảng ánh xạ (mapping) từ ID sang tên để tra cứu label
movies_map_df = movies_df[['movie_id','movie_name']].drop_duplicates('movie_id').reset_index(drop=True)
actors_map_df = actors_df[['actor_id','actor_name']].drop_duplicates('actor_id').reset_index(drop=True)

print(f"- movies_actors: {movies_actors_df.shape} (movie_id, actor_id)")
print(f"- movies: {movies_map_df.shape} (movie_id, movie_name)")
print(f"- actors: {actors_map_df.shape} (actor_id, actor_name)")

display(movies_actors_df.head(10))
display(movies_map_df.head(5))
display(actors_map_df.head(5))

- Dòng có NA: 0
- Dòng trùng: 0
- Tổng số quan hệ phim–diễn viên: 1041
- movies_actors: (1041, 2) (movie_id, actor_id)
- movies: (176, 2) (movie_id, movie_name)
- actors: (569, 2) (actor_id, actor_name)


,movie_id,actor_id
0,M000169,A000425
1,M000169,A000385
2,M000169,A000035
3,M000169,A000259
4,M000169,A000449
5,M000002,A000265
6,M000002,A000255
7,M000002,A000465
8,M000002,A000249
9,M000002,A000502


,movie_id,movie_name
0,M000001,1990
1,M000002,30 chưa phải tết
2,M000003,578: Phát đạn của kẻ điên
3,M000004,Ai thương ai mến
4,M000005,"B4S – Trước giờ ""Yêu"""


,actor_id,actor_name
0,A000001,A Tới
1,A000002,Amee
2,A000003,Anh Dũng
3,A000004,Anh Phạm
4,A000005,Anh Thư


## tạo node

In [16]:
# nodes_df: id, label
nodes_df = actors_df.rename(columns={
    "actor_id": "id",
    "actor_name": "label"
})[["id", "label"]].drop_duplicates().reset_index(drop=True)

print(f"Số node (diễn viên): {len(nodes_df)}")
display(nodes_df.head(10))

Số node (diễn viên): 569


,id,label
0,A000001,A Tới
1,A000002,Amee
2,A000003,Anh Dũng
3,A000004,Anh Phạm
4,A000005,Anh Thư
5,A000006,Anh Tuấn
6,A000007,Anh Tài
7,A000008,Anh Tú
8,A000009,Anh Tú Atus
9,A000010,Anh Tú Wilson


## tạo cạnh

In [17]:
from itertools import combinations
import pandas as pd

#Danh sách diễn viên theo từng phim
cast_by_movie = (
    movies_actors_df
    .drop_duplicates(subset=["movie_id", "actor_id"])
    .groupby("movie_id")["actor_id"]
    .apply(list)
    .reset_index()
)

# Sinh cặp cho từng phim
edge_rows = []
for _, row in cast_by_movie.iterrows():
    movie = row["movie_id"]
    actors_in_movie = row["actor_id"]
    # Nếu phim có <2 diễn viên thì không tạo cạnh
    if len(actors_in_movie) < 2:
        continue
    for a, b in combinations(actors_in_movie, 2):
        # Chuẩn hoá cặp không hướng
        s, t = sorted([a, b])
        edge_rows.append((s, t, 1))

edges_raw = pd.DataFrame(edge_rows, columns=["source", "target", "w"])
print(f"Cặp cộng tác thô (chưa gộp trọng số): {len(edges_raw)}")

# 5.3 — Gộp theo (source, target) để tính weight = số phim chung
edges_df = (
    edges_raw
    .groupby(["source", "target"], as_index=False)["w"]
    .sum()
    .rename(columns={"w": "weight"})
)

# 5.4 — Loại self-loop (phòng hờ) và kiểm tra
edges_df = edges_df[edges_df["source"] != edges_df["target"]].reset_index(drop=True)

print(f"Số cạnh sau gộp trọng số: {len(edges_df)}")
print(f"Thống kê trọng số — min: {edges_df.weight.min()} | max: {edges_df.weight.max()} | mean: {edges_df.weight.mean():.2f}")
display(edges_df.head(10))

Cặp cộng tác thô (chưa gộp trọng số): 3510
Số cạnh sau gộp trọng số: 3405
Thống kê trọng số — min: 1 | max: 6 | mean: 1.03


,source,target,weight
0,A000001,A000023,1
1,A000001,A000107,1
2,A000001,A000262,1
3,A000001,A000310,1
4,A000001,A000363,1
5,A000001,A000472,1
6,A000001,A000477,1
7,A000002,A000395,1
8,A000002,A000398,1
9,A000002,A000456,1


In [20]:
# ==== Bước 6 — Kiểm tra & Xuất nodes.csv, edges.csv ====

import pandas as pd

# 6.1 — Kiểm tra tính toàn vẹn
assert set(['id','label']).issubset(nodes_df.columns), "nodes_df thiếu cột id/label"
assert set(['source','target','weight']).issubset(edges_df.columns), "edges_df thiếu cột source/target/weight"

# Không NA
na_nodes = nodes_df.isna().any(axis=1).sum()
na_edges = edges_df.isna().any(axis=1).sum()

# Source/target phải nằm trong nodes
node_ids = set(nodes_df['id'].tolist())
invalid_edges = edges_df[~edges_df['source'].isin(node_ids) | ~edges_df['target'].isin(node_ids)]

# Không self-loop & không trùng edge (đã gộp trọng số)
self_loops = (edges_df['source'] == edges_df['target']).sum()
dup_edges = len(edges_df) - len(edges_df.drop_duplicates(subset=['source','target']))

print(f"- Node NA rows: {na_nodes}")
print(f"- Edge NA rows: {na_edges}")
print(f"- Self-loops: {self_loops}")
print(f"- Edge duplicates (sau gộp): {dup_edges}")
print(f"- Edges có id không hợp lệ (không có trong nodes): {len(invalid_edges)}")

# 6.2 — (Tuỳ chọn) Lọc theo ngưỡng trọng số để đơn giản hoá đồ thị khi vẽ
MIN_WEIGHT = 1   # đổi thành 2 hoặc 3 nếu muốn bớt rối khi vẽ trong Gephi
edges_to_export = edges_df[edges_df['weight'] >= MIN_WEIGHT].reset_index(drop=True)

print(f"\n== Thống kê ==")
print(f"- Số node: {len(nodes_df)}")
print(f"- Số edge (weight >= {MIN_WEIGHT}): {len(edges_to_export)}")
print(f"- Trọng số: min={edges_to_export['weight'].min()} | max={edges_to_export['weight'].max()} | mean={edges_to_export['weight'].mean():.2f}")

display(nodes_df.head(5))
display(edges_to_export.head(5))

# 6.3 — Xuất CSV (UTF-8 BOM để mở Excel tiếng Việt không lỗi dấu)
nodes_df[['id','label']].to_csv("nodes.csv", index=False, encoding="utf-8-sig")
edges_to_export[['source','target','weight']].to_csv("edges.csv", index=False, encoding="utf-8-sig")


print("- nodes.csv (cột: id, label)")
print("- edges.csv (cột: source, target, weight) — undirected")

- Node NA rows: 0
- Edge NA rows: 0
- Self-loops: 0
- Edge duplicates (sau gộp): 0
- Edges có id không hợp lệ (không có trong nodes): 0

== Thống kê ==
- Số node: 569
- Số edge (weight >= 1): 3405
- Trọng số: min=1 | max=6 | mean=1.03


,id,label
0,A000001,A Tới
1,A000002,Amee
2,A000003,Anh Dũng
3,A000004,Anh Phạm
4,A000005,Anh Thư


,source,target,weight
0,A000001,A000023,1
1,A000001,A000107,1
2,A000001,A000262,1
3,A000001,A000310,1
4,A000001,A000363,1


- nodes.csv (cột: id, label)
- edges.csv (cột: source, target, weight) — undirected


In [21]:
# Nối edges_df với bảng actors để lấy tên diễn viên
edges_named = edges_df.merge(
    actors_df[['actor_id', 'actor_name']], left_on='source', right_on='actor_id'
).rename(columns={'actor_name': 'actor_source'}).drop('actor_id', axis=1)

edges_named = edges_named.merge(
    actors_df[['actor_id', 'actor_name']], left_on='target', right_on='actor_id'
).rename(columns={'actor_name': 'actor_target'}).drop('actor_id', axis=1)

# Sắp xếp theo weight giảm dần
top10_pairs = edges_named.sort_values('weight', ascending=False).head(10)

print("Top 10 cặp diễn viên hợp tác nhiều phim nhất:")
print(top10_pairs[['actor_source', 'actor_target', 'weight']])

Top 10 cặp diễn viên hợp tác nhiều phim nhất:
        actor_source   actor_target  weight
1670  Kiều Minh Tuấn      Thu Trang       6
3191       Thu Trang      Tiến Luật       4
3041      Quốc Khánh      Tuấn Trần       4
3307      Trấn Thành      Tuấn Trần       3
1095      Huỳnh Đông   Ốc Thanh Vân       3
1942        Lê Giang     Trấn Thành       3
1567        Khả Ngân  Long Đẹp Trai       2
1600         Khả Như     Trấn Thành       2
262          Băng Di  NSƯT Hữu Châu       2
1965        Lê Khánh      Tuấn Khải       2


In [26]:
# == BƯỚC 8: TẠO GRAPH & TÍNH CHỈ SỐ MÔ TẢ ==
# Đang làm gì: dựng đồ thị vô hướng có trọng số từ edges_df, thêm đủ node từ nodes_df (kể cả node cô lập),
# rồi tính n, m, degree trung bình, density, số thành phần liên thông, clustering, diameter (trên thành phần lớn nhất).

import networkx as nx

# 1) Dựng graph vô hướng có trọng số
G = nx.Graph()
# Thêm node với nhãn (để tra cứu nhanh)
G.add_nodes_from([(row.id, {"label": row.label}) for _, row in nodes_df.iterrows()])

# Thêm cạnh có weight
for _, r in edges_df.iterrows():
    G.add_edge(r["source"], r["target"], weight=float(r["weight"]))

# 2) Thêm khoảng cách ngắn nhất = 1/weight (để centrality so khớp cách Gephi dùng weight như "độ gần")
for u, v, d in G.edges(data=True):
    w = d.get("weight", 1.0)
    d["inv_weight"] = 1.0 / w if w > 0 else 1.0

# 3) Thống kê mô tả
n = G.number_of_nodes()
m = G.number_of_edges()
avg_degree = (2.0 * m / n) if n > 0 else 0.0
density = nx.density(G)

# Thành phần liên thông
components = sorted(nx.connected_components(G), key=len, reverse=True)
n_components = len(components)
giant = G.subgraph(components[0]).copy() if components else nx.Graph()

# Clustering (unweighted & weighted)
avg_clust = nx.average_clustering(G)
avg_clust_w = nx.average_clustering(G, weight="weight")

# Diameter trên thành phần lớn nhất (không dùng trọng số)
diameter = nx.diameter(giant) if giant.number_of_nodes() > 1 else 0

print("== Thống kê (Python) ==")
print(f"- Nodes: {n}")
print(f"- Edges: {m}")
print(f"- Average degree: {avg_degree:.3f}")
print(f"- Density: {density:.6f}")
print(f"- Connected components: {n_components} (Giant size = {giant.number_of_nodes()})")
print(f"- Average clustering (unweighted): {avg_clust:.4f}")
print(f"- Average clustering (weighted):   {avg_clust_w:.4f}")
print(f"- Diameter (giant component): {diameter}")

== Thống kê (Python) ==
- Nodes: 569
- Edges: 3405
- Average degree: 11.968
- Density: 0.021071
- Connected components: 13 (Giant size = 545)
- Average clustering (unweighted): 0.7824
- Average clustering (weighted):   0.1329
- Diameter (giant component): 7


In [30]:
import networkx as nx

# ===== DỰNG GRAPH VÔ HƯỚNG (giữ weight = số phim chung) =====
G = nx.Graph()
G.add_nodes_from(nodes_df["id"].tolist())
for _, r in edges_df.iterrows():
    G.add_edge(r["source"], r["target"], weight=float(r["weight"]))

# Đồ thị “Gephi”: bỏ node bậc 0 (không có cạnh)
isolates = [n for n, d in G.degree() if d == 0]
G_gephi = G.copy()
G_gephi.remove_nodes_from(isolates)

def summarize_graph(Gx, title):
    n = Gx.number_of_nodes()
    m = Gx.number_of_edges()
    avg_degree = (2*m/n) if n else 0.0
    density = nx.density(Gx)
    comps = sorted(nx.connected_components(Gx), key=len, reverse=True)
    n_components = len(comps)
    giant = Gx.subgraph(comps[0]).copy() if comps else nx.Graph()
    avg_clust = nx.average_clustering(Gx)  # unweighted (giống Gephi mặc định)

    if giant.number_of_nodes() > 1:
        diameter = nx.diameter(giant)
        avg_path_len = nx.average_shortest_path_length(giant)
    else:
        diameter = 0
        avg_path_len = 0.0

    print(f"\n== {title} ==")
    print(f"- Nodes: {n}")
    print(f"- Edges: {m}")
    print(f"- Average degree: {avg_degree:.3f}")
    print(f"- Density: {density:.6f}")
    print(f"- Connected components: {n_components} (giant = {giant.number_of_nodes()})")
    print(f"- Average clustering (unweighted): {avg_clust:.4f}")
    print(f"- Diameter (giant): {diameter}")
    print(f"- Average path length (giant): {avg_path_len:.4f}")

# 1) Toàn bộ đồ thị (có isolates) — sẽ cho 569 nodes
summarize_graph(G, "THỐNG KÊ (toàn bộ đồ thị)")

# 2) Đồ thị như Gephi (loại isolates) — sẽ khớp ~563 nodes, 3405 edges
summarize_graph(G_gephi, "THỐNG KÊ (khớp Gephi: bỏ isolates)")


== THỐNG KÊ (toàn bộ đồ thị) ==
- Nodes: 569
- Edges: 3405
- Average degree: 11.968
- Density: 0.021071
- Connected components: 13 (giant = 545)
- Average clustering (unweighted): 0.7824
- Diameter (giant): 7
- Average path length (giant): 3.1644

== THỐNG KÊ (khớp Gephi: bỏ isolates) ==
- Nodes: 563
- Edges: 3405
- Average degree: 12.096
- Density: 0.021523
- Connected components: 7 (giant = 545)
- Average clustering (unweighted): 0.7907
- Diameter (giant): 7
- Average path length (giant): 3.1644


In [31]:
import networkx as nx
import pandas as pd

# --- 1) Dựng graph khớp Gephi (đã có nodes_df, edges_df như bạn đang dùng) ---
G = nx.Graph()
G.add_nodes_from(nodes_df["id"].tolist())
for _, r in edges_df.iterrows():
    G.add_edge(r["source"], r["target"], weight=float(r["weight"]))

# Bỏ các node cô lập (gephi cũng bỏ khi import edges trước)
isolates = [n for n, d in G.degree() if d == 0]
G_gephi = G.copy()
G_gephi.remove_nodes_from(isolates)

n = G_gephi.number_of_nodes()

# --- 2) BETWEenness CENTRALITY ---
# NetworkX mặc định normalized=True (đưa về 0..1). Gephi hay hiển thị "raw" rất lớn.
# Ta tính CẢ HAI để so: normalized & raw (raw = normalized * ((n-1)(n-2)/2) cho graph vô hướng)
betw_norm = nx.betweenness_centrality(G_gephi, normalized=True)          # 0..1
scale_raw = ((n - 1) * (n - 2)) / 2 if n >= 3 else 1.0
betw_raw  = {u: v * scale_raw for u, v in betw_norm.items()}             # giống kiểu số lớn ở Gephi

# --- 3) CLOSENESS CENTRALITY ---
# NetworkX closeness chuẩn (tính trên các node reachable, có hệ số hiệu chỉnh).
# Gephi dùng định nghĩa tương đương; có thể lệch rất nhỏ do cách hiệu chỉnh thành phần rời.
close = nx.closeness_centrality(G_gephi)  # 0..1 (cao = gần “toàn mạng” hơn)

# --- 4) Gộp kết quả + tên diễn viên ---
id_to_name = dict(actors_df[["actor_id","actor_name"]].values)

cent_df = pd.DataFrame({
    "id": list(G_gephi.nodes()),
    "label": [id_to_name.get(i, i) for i in G_gephi.nodes()],
    "betweenness_raw": [betw_raw[i]  for i in G_gephi.nodes()],
    "betweenness_norm": [betw_norm[i] for i in G_gephi.nodes()],
    "closeness": [close[i] for i in G_gephi.nodes()],
})

# --- 5) Top 10 cho mỗi chỉ số ---
top_betw_raw = cent_df.sort_values("betweenness_raw", ascending=False).head(10)
top_betw_norm = cent_df.sort_values("betweenness_norm", ascending=False).head(10)
top_close     = cent_df.sort_values("closeness", ascending=False).head(10)

print("== Top 10 Betweenness (RAW, kiểu số lớn giống Gephi) ==")
print(top_betw_raw[["label","betweenness_raw"]].to_string(index=False))

print("\n== Top 10 Betweenness (Normalized 0..1) ==")
print(top_betw_norm[["label","betweenness_norm"]].to_string(index=False))

print("\n== Top 10 Closeness (0..1) ==")
print(top_close[["label","closeness"]].to_string(index=False))

# --- 6) Xuất CSV để so bên Gephi (Data Laboratory) ---
cent_df.to_csv("centrality_betweenness_closeness.csv", index=False, encoding="utf-8-sig")

== Top 10 Betweenness (RAW, kiểu số lớn giống Gephi) ==
           label  betweenness_raw
   NSƯT Hữu Châu     16053.505244
      Quốc Cường     11311.741356
   NSND Hồng Vân      9689.500236
      Quang Tuấn      9600.048530
Quách Ngọc Tuyên      6706.935989
      Hứa Vĩ Văn      6432.052757
    Mạc Văn Khoa      5778.444777
       Tuấn Trần      5679.260224
     Thanh Hương      5380.485134
        Trần Lực      5322.479461

== Top 10 Betweenness (Normalized 0..1) ==
           label  betweenness_norm
   NSƯT Hữu Châu          0.101836
      Quốc Cường          0.071756
   NSND Hồng Vân          0.061466
      Quang Tuấn          0.060898
Quách Ngọc Tuyên          0.042546
      Hứa Vĩ Văn          0.040802
    Mạc Văn Khoa          0.036656
       Tuấn Trần          0.036027
     Thanh Hương          0.034131
        Trần Lực          0.033763

== Top 10 Closeness (0..1) ==
           label  closeness
   NSƯT Hữu Châu   0.446251
   NSND Hồng Vân   0.426378
      Quang Tuấn   0.41825